# NN.Pipeline

뉴럴 네트워크 파이프라인

In [1]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

from nnpipeline import *
from nnpipeline.prototype.composition.LinearExponentialComposition import LinearExponentialComposition
from nnpipeline.tools import *

%matplotlib notebook

In [2]:
class ExponentialAutoEncoder(Pipe):
    """
    training 모드에 따라 다르게 동작하는 Exponential Auto Encoder
    지수적 압축/확장을 수행하는 오토인코더이며,
    train 모드일 때 decoded된 값을 반환하고
    eval 모드일 때 encoded된 값을 반환한다.
    """
    def __init__(self, input_size:int, output_size:int,
                 compression_rate:float = 0.618):
        super(ExponentialAutoEncoder, self).__init__()
        self.encoder = LinearExponentialEncoder(input_size, output_size,
                                                compression_rate=compression_rate)
        self.decoder = LinearExponentialDecoder(output_size, input_size,
                                                expansion_rate=1/compression_rate)

    def forward(self, x):
        if self.training:
            x = self.encoder(x)
            x = self.decoder(x)
            return x

        else:
            return self.encoder(x)



In [3]:
eae = ExponentialAutoEncoder(10, 5, 0.618**(1/2))
eae.eval()
eae.training

False

In [4]:
eae.train()
eae.training

True

In [5]:
lee = LinearExponentialEncoder(100, 34)
lee

LinearExponentialEncoder(
  (layers): Sequential(
    (0): Linear(in_features=100, out_features=61, bias=True)
    (1): BatchNorm1d(61, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=61, out_features=37, bias=True)
    (4): BatchNorm1d(37, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Linear(in_features=37, out_features=34, bias=True)
    (7): BatchNorm1d(34, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
  )
)

In [6]:
l1 = LinearExponentialEncoder(100, 35)
l2 = LinearExponentialEncoder(150, 40)
l3 = LinearExponentialEncoder(120, 30)

lec = LinearExponentialComposition([l1, l2, l3], 80)

lec

LinearExponentialComposition(
  (pipes): ModuleList(
    (0): LinearExponentialEncoder(
      (layers): Sequential(
        (0): Linear(in_features=100, out_features=61, bias=True)
        (1): BatchNorm1d(61, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Linear(in_features=61, out_features=37, bias=True)
        (4): BatchNorm1d(37, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU()
        (6): Linear(in_features=37, out_features=35, bias=True)
        (7): BatchNorm1d(35, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (8): ReLU()
      )
    )
    (1): LinearExponentialEncoder(
      (layers): Sequential(
        (0): Linear(in_features=150, out_features=92, bias=True)
        (1): BatchNorm1d(92, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Linear(in_features=92, out_features=56, bias=True)
        (4): BatchNorm1d(56, eps=

In [10]:
# test
x1, x2, x3 = torch.randn(3, 100), torch.randn(3, 150), torch.randn(3, 120)
y = lec(x1, x2, x3)

print(y.shape)

torch.Size([3, 80])
